In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, LSTM
import yfinance as yf

# Download the historical data for Yahoo stock
df = yf.download('AAPL', start='2015-01-01', end='2023-03-08')

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[['Adj Close']].values)


# Create the training and testing datasets
train_data, test_data = train_test_split(scaled_data, test_size=0.2, shuffle=False)

# Create the input and output sequences for the LSTM model
x_train, y_train = [], []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

x_test, y_test = [], []
for i in range(len(scaled_data)-60, len(scaled_data)):
    x_test.append(scaled_data[i-60:i, 0])
    y_test.append(scaled_data[i, 0])
    
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Create the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25)) 
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
model.evaluate(x_test, y_test)

# Make predictions
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

# Plot the predictions
plt.plot(df[len(df)-len(predictions):]['Adj Close'].values, label="Actual Direction")
plt.plot(predictions, label="Predicted Direction")
plt.legend()
plt.title('AAPL Price Prediction')
plt.show()


In [ ]:
# Create a DataFrame to store the predicted and actual stock prices
df_pred = pd.DataFrame(predictions, columns=['Predicted'], index=df.tail(len(predictions)).index)
df_pred['Actual'] = df['Adj Close'].tail(len(predictions)).values

from sklearn.metrics import mean_squared_error

# Calculate MSE and RMSE
mse = mean_squared_error(df_pred['Actual'], df_pred['Predicted'])
rmse = np.sqrt(mse)
print("MSE: ", mse)
print("RMSE: ", rmse)

# Plot the predicted and actual stock prices
plt.plot(df_pred['Actual'], label="Actual Direction")
plt.plot(df_pred['Predicted'], label="Backtest Direction")
plt.legend()
plt.title('AAPL Back-Test')
plt.show()


In [ ]:
import talib
import yfinance as yf

# Download historical price data for a stock
df = yf.download('AAPL', start='2015-01-01', end='2023-03-01')

# Calculate CCI with a time period of 30
df['cci'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=30)

# Define a function to generate trading signals
def generate_signals(data):
    buy_signal = (data['cci'].shift(1) < -100) & (data['cci'] >= -100)
    sell_signal = (data['cci'].shift(1) > 100) & (data['cci'] <= 100)
    data['signal'] = np.where(buy_signal, 1, np.nan)
    data['signal'] = np.where(sell_signal, -1, data['signal'])
    return data

# Generate trading signals
df = generate_signals(df)

# Add a confirmation filter (e.g., 50-day moving average)
df['ma50'] = df['Close'].rolling(window=50).mean()
df['signal'] = np.where((df['Close'] > df['ma50']) & (df['signal'] == 1), 1, df['signal'])
df['signal'] = np.where((df['Close'] < df['ma50']) & (df['signal'] == -1), -1, df['signal'])

# Plot the signals
plt.figure(figsize=(10,5))
plt.plot(df['Close'], label='Price')
plt.scatter(df[df['signal']==1].index, df[df['signal']==1]['Close'], label='Buy', color='green', marker='^', lw=3)
plt.scatter(df[df['signal']==-1].index, df[df['signal']==-1]['Close'], label='Sell', color='red', marker='v', lw=3)
plt.legend()
plt.title('AAPL Price Chart with CCI(30) Trading Signals')
plt.show()


In [ ]:
import talib
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# Download historical price data for a stock
df = yf.download('AAPL', start='2015-01-01', end='2023-03-01')

# Calculate CCI with a time period of 60
df['cci'] = talib.CCI(df['High'], df['Low'], df['Close'], timeperiod=60)

# Define a function to generate trading signals
def generate_signals(data):
    buy_signal = (data['cci'].shift(1) < -100) & (data['cci'] >= -100)
    sell_signal = (data['cci'].shift(1) > 100) & (data['cci'] <= 100)
    data['signal'] = np.where(buy_signal, 1, np.nan)
    data['signal'] = np.where(sell_signal, -1, data['signal'])
    return data

# Generate trading signals
df = generate_signals(df)

# Add a confirmation filter (e.g., 50-day moving average)
df['ma50'] = df['Close'].rolling(window=50).mean()
df['signal'] = np.where((df['Close'] > df['ma50']) & (df['signal'] == 1), 1, df['signal'])
df['signal'] = np.where((df['Close'] < df['ma50']) & (df['signal'] == -1), -1, df['signal'])

# Plot the signals
plt.figure(figsize=(10,5))
plt.plot(df['Close'], label='Price')
plt.scatter(df[df['signal']==1].index, df[df['signal']==1]['Close'], label='Buy', color='green', marker='^', lw=3)
plt.scatter(df[df['signal']==-1].index, df[df['signal']==-1]['Close'], label='Sell', color='red', marker='v', lw=3)
plt.legend()
plt.title('AAPL Price Chart with CCI(60) Trading Signals')
plt.show()


In [ ]:
import talib
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

# Download historical price data for a stock
df = yf.download('AAPL', start='2015-01-01', end='2023-03-01')

# Calculate RSI with a time period of 14
df['rsi'] = talib.RSI(df['Close'], timeperiod=14)

# Define a function to generate trading signals
def generate_signals(data):
    buy_signal = (data['rsi'].shift(1) < 30) & (data['rsi'] >= 30)
    sell_signal = (data['rsi'].shift(1) > 70) & (data['rsi'] <= 70)
    data['signal'] = np.where(buy_signal, 1, np.nan)
    data['signal'] = np.where(sell_signal, -1, data['signal'])
    return data

# Generate trading signals
df = generate_signals(df)

# Add a confirmation filter (e.g., 50-day moving average)
df['ma50'] = df['Close'].rolling(window=50).mean()
df['signal'] = np.where((df['Close'] > df['ma50']) & (df['signal'] == 1), 1, df['signal'])
df['signal'] = np.where((df['Close'] < df['ma50']) & (df['signal'] == -1), -1, df['signal'])

# Plot the signals
plt.figure(figsize=(10,5))
plt.plot(df['Close'], label='Price')
plt.scatter(df[df['signal']==1].index, df[df['signal']==1]['Close'], label='Buy', color='green', marker='^', lw=3)
plt.scatter(df[df['signal']==-1].index, df[df['signal']==-1]['Close'], label='Sell', color='red', marker='v', lw=3)
plt.legend()
plt.title('AAPL Price Chart with RSI Trading Signals')
plt.show()
